# Regularization

## L1 and L2

* The first form of regularization are L1 and L2 and these are used to calculate the value `penalty` so that it can be used to calculate with the `loss` value to make the generalization error less
* L1's regularization penalty is the sum of all the absolute values of weights and bias. It is linear penalty as it is directly proportional to the weights and bias
* L2's regularization penalty is the sum of squared weights and bias. It is non linear and it is commonly used as it doesn't affect smaller valued parameters and larger valued parameters to grow even bigger
* We use $\lambda$ in the equation if it is bigger then the penalty will be bigger

$\text{Loss} = \text{data loss} + \text{L1 for weights} + \text{L1 for bias} + \text{L2 for weights} + \text{L2 for bias}$


In [6]:
import numpy as np
from tqdm import tqdm
import nnfs
from nnfs.datasets import spiral_data

nnfs.init()

In [3]:
class Layer_Dense:
    def __init__(self,n_inputs,n_neurons,
                 weight_regularizer_l1 = 0,weight_regularizer_l2=0,
                 bias_regularizer_l1 = 0, bias_regularizer_l2 = 0):
        # init of weights and bias
        self.weights = 0.01 * np.random.randn(n_inputs,n_neurons)
        self.biases = np.zeros((1,n_neurons))
        # init the regularization
        self.weight_regularizer_l1 = weight_regularizer_l1
        self.weight_regularizer_l2 = weight_regularizer_l2
        self.bias_regularizer_l1 = bias_regularizer_l1
        self.bias_regularizer_l2 = bias_regularizer_l2
        
        
    def forward(self,inputs):
        self.inputs = inputs
        self.output = np.dot(inputs,self.weights) + self.biases
        
    def backward(self,dvalues):
        # calculating gradients for parameters
        self.dweights = np.dot(self.inputs.T,dvalues)
        self.dbiases = np.sum(dvalues,axis=0,keepdims=True)
        
        # calculating gradients for regularization
        if self.weight_regularizer_l1 > 0:
            dL1 = np.ones_like(self.weights)
            dL1[self.weights < 0] = -1
            self.dweights += self.weight_regularizer_l1 * dL1
        if self.weight_regularizer_l2 > 0:
            self.dweights += 2 * self.weight_regularizer_l2 * self.weights
        
        if self.bias_regularizer_l1 > 0:
            dL1 = np.ones_like(self.biases)
            dL1[self.biases < 0] = -1
            self.dbiases += self.bias_regularizer_l1 * dL1
        if self.bias_regularizer_l2 > 0:
            self.dbiases += 2 * self.bias_regularizer_l2 * self.biases 
            
        # gradients on whole values
        self.dinputs = np.dot(dvalues,self.weights.T)

In [ ]:
class Loss:
    def regularization_loss(self,layer):
        regularization_loss = 0
        # Weight Loss
        if layer.weight_regularizer_l1 > 0:
            # sum of absolute values of weights
            regularization_loss += layer.weight_regularizer_l1 * np.sum(np.abs(layer.weights))
        if layer.weight_regularizer_l2 > 0:
            # sum of squared values of weights
            regularization_loss += layer.weight_regularizer_l2 * np.sum(layer.weights * layer.weights)
        if layer.bias_regularizer_l1 > 0:
            regularization_loss += layer.bias_regularizer_l1 * np.sum(np.abs(layer.biases))
        if layer.bias_regularizer_l2 > 0:
            regularization_loss += layer.bias_regularizer_l2 * np.sum(layer.biases * layer.biases)
            
        return regularization_loss
    
    def calculate(self,output,y):
        sample_losses = self.forward(output,y)
        data_loss = np.mean(sample_losses)
        return data_loss

class Loss_CategoricalCrossentropy(Loss):
    def forward(self,y_pred,y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred,1e-7,1 - 1e-7)
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples),y_true]
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true,axis=1)
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods
    
    def backward(self,dvalues,y_true):
        samples = len(dvalues)
        labels = len(dvalues[0])
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]
        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples

In [5]:
class Activation_RELU:
    def forward(self,inputs):
        self.inputs = inputs
        self.output = np.maximum(0,inputs)
        
    def backward(self,dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0
        
class Activation_Softmax:
    def forward(self,inputs):
        self.inputs = inputs
        exp_values = np.exp(inputs - np.max(inputs,axis=1,keepdims=True))
        self.output = exp_values / np.sum(exp_values,axis=1,keepdims=True)
        
    def backward(self,dvalues):
        self.dinputs = np.empty_like(dvalues)
        for index, (single_output,single_dvalues) in enumerate(zip(self.output,dvalues)):
            single_output = single_output.reshape(-1,1)
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output,single_output.T)
            self.dinputs[index] = np.dot(jacobian_matrix,single_dvalues) 
    
class Activation_Softmax_Loss_CategoricalCrossentropy:
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossentropy()
    
    def forward(self,inputs,y_true):
        self.activation.forward(inputs)
        self.output = self.activation.output
        return self.loss.calculate(self.output,y_true)
    
    def backward(self,dvalues,y_true):
        samples = len(dvalues)
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true,axis=1)
        self.dinputs = dvalues.copy()
        self.dinputs[range(samples),y_true] -= 1
        self.dinputs = self.dinputs / samples
        
class Optimizer_SGD:
    def __init__(self,learning_rate=1.,decay=0.,momentum=0.):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum
        
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))
    
    def update_params(self,layer):
        if self.momentum:
            if not hasattr(layer,'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)
            # past - future direction
            weight_updates = self.momentum * layer.weight_momentums - self.current_learning_rate * layer.dweights
            layer.weight_momentums = weight_updates
            bias_updates = self.momentum * layer.bias_momentums - self.current_learning_rate * layer.dbiases
            layer.bias_momentums = bias_updates
        else:
            weight_updates = -self.current_learning_rate * layer.dweights
            bias_updates = -self.current_learning_rate * layer.dbiases
        layer.weights += weight_updates
        layer.biases += bias_updates
    
    def post_update_params(self):
        self.iterations += 1  

class Optimizer_Adagrad:
    def __init__(self,learning_rate=1.,decay=0.,epsilon=1e-7):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))
            
    def update_params(self,layer):
        if not hasattr(layer,'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)
        layer.weight_cache += layer.dweights**2
        layer.bias_cache += layer.dbiases**2
        layer.weights += -self.current_learning_rate * layer.dweights / (np.sqrt(layer.weight_cache) + self.epsilon)
        layer.biases += -self.current_learning_rate * layer.dbiases / np.sqrt(layer.bias_cache) + self.epsilon
    
    def post_update_params(self):
        self.iterations += 1
        
class Optimizer_RMSprop:
    def __init__(self,learning_rate=0.001,decay=0.,epsilon=1e-7,rho=0.9):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.rho = rho
        
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))
            
    def update_params(self,layer):
        if not hasattr(layer,'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)
        layer.weight_cache = self.rho * layer.weight_cache + (1 - self.rho) * layer.dweights ** 2
        layer.bias_cache = self.rho * layer.bias_cache + ( 1 - self.rho) * layer.dbiases ** 2
        layer.weights += -self.current_learning_rate * layer.dweights / (np.sqrt(layer.weight_cache) + self.epsilon)
        layer.biases += -self.current_learning_rate * layer.dbiases / (np.sqrt(layer.bias_cache) + self.epsilon)
        
    def post_update_params(self):
        self.iterations += 1
        
class Optimizer_Adam:
    def __init__(self, learning_rate=0.001, decay=0., epsilon=1e-7, beta_1=0.9, beta_2=0.999):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.beta_1 = beta_1
        self.beta_2 = beta_2
    
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))
    
    def update_params(self, layer):
        if not hasattr(layer, 'weight_cache'):
            layer.weight_momentums = np.zeros_like(layer.weights)
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_momentums = np.zeros_like(layer.biases)
            layer.bias_cache = np.zeros_like(layer.biases)
        layer.weight_momentums = self.beta_1 * layer.weight_momentums + (1 - self.beta_1) * layer.dweights
        layer.bias_momentums = self.beta_1 * layer.bias_momentums + (1 - self.beta_1) * layer.dbiases
        weight_momentums_corrected = layer.weight_momentums / (1 - self.beta_1 ** (self.iterations + 1))
        bias_momentums_corrected = layer.bias_momentums / (1 - self.beta_1 ** (self.iterations + 1))
        layer.weight_cache = self.beta_2 * layer.weight_cache + (1 - self.beta_2) * layer.dweights**2
        layer.bias_cache = self.beta_2 * layer.bias_cache + (1 - self.beta_2) * layer.dbiases**2
        weight_cache_corrected = layer.weight_cache / (1 - self.beta_2 ** (self.iterations + 1))
        bias_cache_corrected = layer.bias_cache / (1 - self.beta_2 ** (self.iterations + 1))
        layer.weights += -self.current_learning_rate * weight_momentums_corrected / (np.sqrt(weight_cache_corrected) + self.epsilon)
        layer.biases += -self.current_learning_rate * bias_momentums_corrected / (np.sqrt(bias_cache_corrected) + self.epsilon)
    
    def post_update_params(self):
        self.iterations += 1

In [10]:
X, y = spiral_data(samples=100, classes=3)

dense1 = Layer_Dense(2, 64,weight_regularizer_l2=5e-4,bias_regularizer_l2=5e-4)
activation1 = Activation_RELU()
dense2 = Layer_Dense(64, 3)
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()
optimizer = Optimizer_Adam(learning_rate=0.05, decay=5e-7)
epoches = []
accuracies = []
losses = []
learning_rates = []

In [ ]:
# Training loop
for epoch in tqdm(range(5001),desc="Training the model"):
    # Forward pass
    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    
    # Loss calculation
    data_loss = loss_activation.forward(dense2.output, y)
    regularization_loss = loss_activation.loss.regularization_loss(dense1) + loss_activation.loss.regularization_loss(dense2)
    loss = data_loss + regularization_loss
    
    # Calculate accuracy
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions == y)
    
    # Print progress every 100 epochs
    if not epoch % 100:
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy:.3f}, ' +
              f'loss: {loss:.3f}, ' +
              f'lr: {optimizer.current_learning_rate}')
        epoches.append(epoch)
        accuracies.append(accuracy)
        losses.append(loss)
        learning_rates.append(optimizer.current_learning_rate)
    
    # Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    
    # Update parameters
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

Training the model:   4%|▍         | 225/5001 [00:00<00:04, 1098.40it/s]

epoch: 0, acc: 0.343, loss: 1.099, lr: 0.05
epoch: 100, acc: 0.757, loss: 0.724, lr: 0.04999752512250644
epoch: 200, acc: 0.800, loss: 0.574, lr: 0.04999502549496326


Training the model:  10%|█         | 504/5001 [00:00<00:03, 1296.48it/s]

epoch: 300, acc: 0.843, loss: 0.518, lr: 0.049992526117345455
epoch: 400, acc: 0.867, loss: 0.491, lr: 0.04999002698961558
epoch: 500, acc: 0.860, loss: 0.450, lr: 0.049987528111736124


Training the model:  16%|█▌        | 778/5001 [00:00<00:03, 1336.84it/s]

...

epoch: 4200, acc: 0.917, loss: 0.275, lr: 0.04989524493326262
epoch: 4300, acc: 0.917, loss: 0.274, lr: 0.04989275552200545
epoch: 4400, acc: 0.917, loss: 0.271, lr: 0.04989026635914307


Training the model:  93%|█████████▎| 4668/5001 [00:03<00:00, 1146.36it/s]

epoch: 4500, acc: 0.920, loss: 0.269, lr: 0.04988777744463829
epoch: 4600, acc: 0.917, loss: 0.270, lr: 0.049885288778453954
epoch: 4700, acc: 0.923, loss: 0.265, lr: 0.049882800360552884


Training the model: 100%|██████████| 5001/5001 [00:03<00:00, 1262.33it/s]

epoch: 4800, acc: 0.917, loss: 0.263, lr: 0.04988031219089794
epoch: 4900, acc: 0.917, loss: 0.264, lr: 0.049877824269451976
epoch: 5000, acc: 0.920, loss: 0.281, lr: 0.04987533659617785


In [13]:
X_test, y_test = spiral_data(samples=100, classes=3)
dense1.forward(X_test)
activation1.forward(dense1.output)
dense2.forward(activation1.output) 
loss = loss_activation.forward(dense2.output, y_test)
predictions = np.argmax(loss_activation.output, axis=1) 
if len(y_test.shape) == 2: 
    y_test = np.argmax(y_test, axis=1) 
accuracy = np.mean(predictions==y_test)
print(f'validation, acc: {accuracy:.3f}, loss: {loss:.3f}') 

validation, acc: 0.817, loss: 0.532


## Dropout

In [15]:
class Layer_Dropout:
    def __init__(self,rate):
        self.rate = 1 - rate
        
    def forward(self,inputs):
        self.inputs = inputs
        self.binary_mask = np.random.binomial(1,self.rate,size=inputs.shape) / self.rate
        self.output = inputs * self.binary_mask
        
    def backward(self,dvalues):
        self.dinputs = dvalues * self.binary_mask

In [20]:
dense1 = Layer_Dense(2, 512,weight_regularizer_l2=5e-4,bias_regularizer_l2=5e-4)
activation1 = Activation_RELU()
dropout_1 = Layer_Dropout(0.1)
dense2 = Layer_Dense(512, 3)
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()
optimizer = Optimizer_Adam(learning_rate=0.05, decay=5e-5)

In [21]:
# Training loop
for epoch in tqdm(range(5001),desc="Training the model"):
    # Forward pass
    dense1.forward(X)
    activation1.forward(dense1.output)
    dropout_1.forward(activation1.output)
    dense2.forward(dropout_1.output)
    
    # Loss calculation
    data_loss = loss_activation.forward(dense2.output, y)
    regularization_loss = loss_activation.loss.regularization_loss(dense1) + loss_activation.loss.regularization_loss(dense2)
    loss = data_loss + regularization_loss
    
    # Calculate accuracy
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions == y)
    
    # Print progress every 100 epochs
    if not epoch % 100:
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy:.3f}, ' +
              f'loss: {loss:.3f}, ' +
              f'lr: {optimizer.current_learning_rate}')
        epoches.append(epoch)
        accuracies.append(accuracy)
        losses.append(loss)
        learning_rates.append(optimizer.current_learning_rate)
    
    # Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    dropout_1.backward(dense2.dinputs)
    activation1.backward(dropout_1.dinputs)
    dense1.backward(activation1.dinputs)
    
    # Update parameters
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

Training the model:   0%|          | 17/5001 [00:00<01:02, 80.05it/s]

epoch: 0, acc: 0.297, loss: 1.099, lr: 0.05


Training the model:   2%|▏         | 116/5001 [00:01<00:59, 82.56it/s]

epoch: 100, acc: 0.693, loss: 0.729, lr: 0.04975371909050202


Training the model:   4%|▍         | 215/5001 [00:02<00:56, 85.29it/s]

...

epoch: 4800, acc: 0.863, loss: 0.436, lr: 0.04032420662123473


Training the model:  98%|█████████▊| 4914/5001 [01:05<00:01, 83.63it/s]

epoch: 4900, acc: 0.837, loss: 0.594, lr: 0.04016225551226957


Training the model: 100%|██████████| 5001/5001 [01:06<00:00, 75.50it/s]

epoch: 5000, acc: 0.837, loss: 0.511, lr: 0.04000160006400256


In [22]:
X_test, y_test = spiral_data(samples=100, classes=3)
dense1.forward(X_test)
activation1.forward(dense1.output)
dropout_1.forward(activation1.output)
dense2.forward(dropout_1.output) 
loss = loss_activation.forward(dense2.output, y_test)
predictions = np.argmax(loss_activation.output, axis=1) 
if len(y_test.shape) == 2: 
    y_test = np.argmax(y_test, axis=1) 
accuracy = np.mean(predictions==y_test)
print(f'validation, acc: {accuracy:.3f}, loss: {loss:.3f}') 

validation, acc: 0.767, loss: 0.861
